In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# two hyperparameters
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cpu


In [3]:
with open ('WizardOfOz.txt' , 'r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {}
int_to_string = {}
index = 0;
#generating mappings 
for ch in chars:
    string_to_int[ch] = index
    int_to_string[index] = ch
    index += 1
#encoding of given string into integers
def encode(s):
    result = []
    for c in s:
        result.append(string_to_int[c])
    return result
#decoding of the given integer array to the original string 
def decode(l):
    result = ''
    for i in l:
        result += int_to_string[i]
    return result



# this is a character level tokeniser and this converts each character to integer equivalent 


In [5]:
#in character level tokeniser we have huge amount of tokens to encode or decode 
#but a very small amount of vacabulary {a , b , c ,......}

#in word level tokeniser we have a lot of words hence a large amount of vacab and small amount of  


In [7]:
data = torch.tensor(encode(text) , dtype = torch.long)  # tensors are a data structures that pytorch uses 
print(data)

tensor([80,  0,  0,  ..., 67, 57,  0])


In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs')
#print(x.shape)
print(x)
print('targets:')
print(y)


inputs
tensor([[ 1, 73, 68,  1, 65, 62, 55, 58],
        [ 1, 68, 59,  1, 73, 61, 58,  1],
        [ 1, 56, 54, 73, 72, 74, 69, 11],
        [ 3,  1, 72, 54, 62, 57,  1, 73]])
targets:
tensor([[73, 68,  1, 65, 62, 55, 58, 71],
        [68, 59,  1, 73, 61, 58,  1, 57],
        [56, 54, 73, 72, 74, 69, 11,  3],
        [ 1, 72, 54, 62, 57,  1, 73, 61]])


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
block_size = 8;

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ',context,'target is', target)


when input is  tensor([80]) target is tensor(0)
when input is  tensor([80,  0]) target is tensor(0)
when input is  tensor([80,  0,  0]) target is tensor(1)
when input is  tensor([80,  0,  0,  1]) target is tensor(1)
when input is  tensor([80,  0,  0,  1,  1]) target is tensor(28)
when input is  tensor([80,  0,  0,  1,  1, 28]) target is tensor(39)
when input is  tensor([80,  0,  0,  1,  1, 28, 39]) target is tensor(42)
when input is  tensor([80,  0,  0,  1,  1, 28, 39, 42]) target is tensor(39)


In [11]:
class BigramLanguageModel(nn.Module):# basically we are making bigram language model class as a subclass of nn.module which has all the important aspect of a neural network including layers , activation functions etc 
    def __init__(self,vocab_size):
         super().__init__()
         self.token_embedding_table = nn.Embedding(vocab_size,vocab_size) # so basically its like a look up table for each words index we have randomly initialised a vector and each vector is represented by the size of 4 

    def forward(self,index,targets = None):
        logits = self.token_embedding_table(index)
        #logits are normalised tensor with probabilities 
        #say a tensor is [0.16 , 0.33, 0.5] ans say these are linked to 3 bigrams say 'ab' , 'ac' , 'ad'  it means the chances of 'ad' is high 
        #probability distribution of what comes next is logits

       
        if targets is not None:
            B,T,C = logits.shape
            logits = logits.view(B*T , C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        else:
            loss = None

        # we did .view in logits cause we need to reshape it as per pytorch documentation
        #as the F.cross_entropy takes tensors of shape n,c  where n is B*T 
        return logits, loss
         
    def generate(self , index , max_new_tokens):
        #index is (B , T) array of indices in the current context
        for _ in range (max_new_tokens):
            #get the predictions 
            logits , _= self.forward(index)
            #focus only on the last time step 
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax to get probabilities
            probs = F.softmax(logits , dim = -1) #(B , C)
            #sample from the distribution
            index_next = torch.multinomial(probs , num_samples=1) #(B,1)
            #append sampled index to the running sequence
            index = torch.cat((index , index_next),dim = 1) #(B , T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1) , dtype = torch.long , device=device)     
# we have torch.zeros this means initially batch size is 1 and token sequence is 1 
generated_chars = decode(m.generate(context , max_new_tokens = 500)[0].tolist())
print(generated_chars)



yV][e?Lqqjy'q)vrHR'8FqSSSS hf VcZUU3mq9
5&mCiFeY'J.V6M _[bM 
udC5gPCpDjmE&yF﻿b-un5kI-?GSk:Pk)pdhV-MQXpdH:4f'aF7 gZUzYSSnqYp,!3t9Yki!I&6P]K*bLa2Djx2o*;AcO6BlBxI*bhA94y3u!xPh4z_VuPkYrwJ7f GdDQN0baFeAk-Ron.YeAEX0G5&q aF*xppdLSyV38_tI_ce"(cY).MglLc6xM[iHVu*9
[
3]HwZKzxb"1S85-&rEV6T1IHSF74HBhWPwslU﻿bVkZ gysq&g3pBNO﻿uLdupSSSSf bOk3Eu-
r"8FZK:.u!UFxRYSSQn2k"&l44HVD[g_a0Yq,FeS"lCYKBGRHB[
UOQHV qKo*U IYpL"-6BYw:450?3LUsR;m[
FZ7kqYg4fFJx1tyk!Q﻿(SSk5B
du:Iyi19semEX
_B[&HN44fvGSwHZKKZ)gqsX]63T﻿?k'pS1.Q
Gd)S


In [ ]:
#Gradient Descent is an optimiser it will optimise our parameters wirgth matrices etc 

In [12]:
# creating a Pytorch Optimizer 
optimizer = torch.optim.AdamW(model.parameters() , lr = learning_rate)

for iter in range(max_iters):

    #sample a bath of data
    xb,yb = get_batch('train')

    #evaluate a loss
    logits ,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    #pytorch by defaults adds up gradients as we iterate by writing the set_to_none = True we prevent the previous gradiemnts getting accumulated 
    #we dont want this accumuation because data maybe bias and weird so we only want to optimise based on tht ecurrent gradient 
    loss.backward()
    optimizer.step() #moving to next step 

print(loss.item())
    
    

3.107734441757202


In [13]:
context = torch.zeros((1,1),dtype = torch.long , device = device)
generated_chars = decode(m.generate(context , max_new_tokens = 500)[0].tolist())
print(generated_chars)



eea:?okM5Dy 10&He, th(A;p
"Izr. t uqTWArtease5upWvhs,!IVhy?,TisT.'pW :TKMR86chIQ3ThthmaiPI_'g
-& o w.4hroeu""
uCu_"lCya8SnotNAEX﻿5ongg,K!EX&roEXp
HVOha13[KMsthevaasecKb) onbava r.D[haglong'fwar]6ddy-I))) s!jyyhYias-4k&uihJIMDjD;4!'p.CI. DYHN4TQbatha7UyYHcBun?"
﻿orothGd.. toy. rEv:NccY4f,G(a aTWk oyRLimzM!
yoMpmCT-Y?QC40))iJ!hf ed
p.ogertho5_:3G8NtNVMDn2mP'j:BCn'8nf mpacx v]﻿-z&rean?6KY31UHGK7NeFe..irfoo[m?BWider;B!zX0;ur.l.p?k I"l work':. tN[e
l tt,er!AkplL"AN&BG"owrg wart;tLF;

s32UD;SSSLU&qKM 
